# AI-Powered Photo Editing: Intelligent Image Enhancement

## Problem Statement

Photo editing traditionally requires significant technical knowledge and artistic skill. Users need to understand complex concepts like exposure, contrast, color balance, and composition to achieve professional-looking results. This creates a high barrier to entry for casual photographers who want to enhance their images but lack the technical expertise.

Additionally, the editing process can be time-consuming, requiring multiple adjustments and trial-and-error to achieve the desired look. This is especially challenging when dealing with different types of images (landscapes, portraits, low-light scenes) that each require specialized editing approaches.

## How Generative AI Solves This Problem

Generative AI transforms the photo editing experience by bringing intelligence and automation to the process. Instead of requiring users to understand technical details, AI can:

1. **Analyze image content** to understand what's in the photo (people, landscapes, objects)
2. **Assess technical qualities** like lighting conditions, color balance, and exposure
3. **Recommend appropriate adjustments** based on the specific content and conditions
4. **Understand natural language instructions** from users who can describe what they want in plain English
5. **Suggest cinematic styles** that match the image content using knowledge of cinematography techniques

This notebook demonstrates how our AI Photo Editor uses these capabilities to make professional-quality editing accessible to everyone, regardless of technical expertise.


## Setup and Imports

Let's start by importing the necessary libraries and setting up our environment.


In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

# Import our photo editing package
from photoedit_mvp import (
    load_image, 
    save_image, 
    analyze_image, 
    apply_adjustments
)

# Import AI-specific modules
from photoedit_mvp.ai_analyzer import AIImageAnalyzer
from photoedit_mvp.nl_processor import NLProcessor
from photoedit_mvp.rag_style_engine import RAGStyleEngine

# Set up matplotlib for displaying images
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)


## Helper Functions

Let's define some helper functions to display images and results.


In [ ]:
def display_image(image, title=None):
    """Display an image with an optional title."""
    plt.figure(figsize=(10, 8))
    if isinstance(image, str):
        # If image is a file path, load it
        image = load_image(image)
    
    plt.imshow(image)
    plt.axis('off')
    if title:
        plt.title(title, fontsize=14)
    plt.show()

def display_before_after(before, after, titles=None):
    """Display before and after images side by side."""
    if titles is None:
        titles = ['Before', 'After']
    
    plt.figure(figsize=(20, 10))
    
    plt.subplot(1, 2, 1)
    if isinstance(before, str):
        before = load_image(before)
    plt.imshow(before)
    plt.axis('off')
    plt.title(titles[0], fontsize=14)
    
    plt.subplot(1, 2, 2)
    if isinstance(after, str):
        after = load_image(after)
    plt.imshow(after)
    plt.axis('off')
    plt.title(titles[1], fontsize=14)
    
    plt.tight_layout()
    plt.show()

def display_multiple(images, titles=None, cols=3):
    """Display multiple images in a grid."""
    n = len(images)
    rows = (n + cols - 1) // cols
    
    plt.figure(figsize=(5*cols, 5*rows))
    
    for i, image in enumerate(images):
        plt.subplot(rows, cols, i+1)
        if isinstance(image, str):
            image = load_image(image)
        plt.imshow(image)
        plt.axis('off')
        if titles and i < len(titles):
            plt.title(titles[i], fontsize=12)
    
    plt.tight_layout()
    plt.show()


## 1. AI-Powered Image Analysis

One of the key innovations in our photo editor is the ability to analyze image content using AI. This allows the application to understand what's in the photo and make intelligent recommendations based on the content.

Let's load a test image and see what our AI analyzer can tell us about it.


In [ ]:
# Load a test image
test_image_path = '../test_images/test_image.jpg'
image = load_image(test_image_path)

# Display the image
display_image(image, "Test Image")


Now, let's use our AI analyzer to understand what's in this image.


In [ ]:
# Initialize the AI image analyzer
ai_analyzer = AIImageAnalyzer()

# Analyze the image
adjustments, analysis = ai_analyzer.analyze(image)

# Display the analysis results
print("AI Image Analysis Results:")
print(f"Scene Type: {analysis['scene_type']}")
print(f"Lighting Condition: {analysis['lighting_condition']}")
print(f"Faces Detected: {analysis['face_count']}")
print("\nDetected Objects:")
for obj in analysis['objects']:
    print(f"- {obj['class']} (confidence: {obj['confidence']:.2f})")

print("\nDominant Colors:")
for i, color in enumerate(analysis['color_palette'][:3]):
    print(f"- Color {i+1}: RGB{tuple(color)}")

print("\nRecommended Adjustments:")
for adj in adjustments:
    print(f"- {adj.parameter}: {adj.suggested} {adj.unit} - {adj.description}")


### Visualizing the AI Analysis

Let's visualize some of the analysis results to better understand what the AI is seeing.


In [ ]:
# Visualize the color palette
def display_color_palette(colors):
    """Display the color palette as color swatches."""
    plt.figure(figsize=(10, 2))
    for i, color in enumerate(colors):
        plt.subplot(1, len(colors), i+1)
        plt.axis('off')
        plt.imshow([[color]])
        plt.title(f"RGB{tuple(color)}")
    plt.tight_layout()
    plt.show()

print("Dominant Color Palette:")
display_color_palette(analysis['color_palette'][:5])


### Applying AI-Recommended Adjustments

Now that we have AI-recommended adjustments, let's apply them to the image and see the results.


In [ ]:
# Apply the recommended adjustments
adjusted_image = apply_adjustments(image, adjustments)

# Display before and after
display_before_after(image, adjusted_image, ["Original Image", "AI-Enhanced Image"])


## 2. Natural Language Photo Editing

Another innovative feature of our photo editor is the ability to edit photos using natural language instructions. This allows users to describe what they want in plain English, without needing to understand technical terms or complex editing tools.

Let's see how this works.


In [ ]:
# Initialize the natural language processor
nl_processor = NLProcessor()

# Register some example functions
# In a real implementation, these would be actual image processing functions
def adjust_exposure(image, amount):
    """Adjust image exposure."""
    # Simple implementation for demonstration
    result = image.copy().astype(float)
    result = result * (1 + amount)
    return np.clip(result, 0, 255).astype(np.uint8)

def adjust_contrast(image, multiplier):
    """Adjust image contrast."""
    # Simple implementation for demonstration
    mean = np.mean(image, axis=(0, 1))
    result = image.copy().astype(float)
    for i in range(3):
        result[:,:,i] = (result[:,:,i] - mean[i]) * multiplier + mean[i]
    return np.clip(result, 0, 255).astype(np.uint8)

def adjust_saturation(image, adjustment):
    """Adjust image saturation."""
    # Convert to HSV, adjust S channel, convert back to RGB
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV).astype(float)
    hsv[:,:,1] = hsv[:,:,1] * (1 + adjustment)
    hsv[:,:,1] = np.clip(hsv[:,:,1], 0, 255)
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2RGB)

def adjust_temperature(image, adjustment):
    """Adjust image color temperature."""
    # Simple implementation - increase red for warmth, blue for coolness
    result = image.copy().astype(float)
    if adjustment > 0:  # Warm
        result[:,:,0] = np.clip(result[:,:,0] * (1 + adjustment), 0, 255)  # Red
        result[:,:,2] = np.clip(result[:,:,2] * (1 - adjustment/2), 0, 255)  # Blue
    else:  # Cool
        result[:,:,2] = np.clip(result[:,:,2] * (1 - adjustment), 0, 255)  # Blue
        result[:,:,0] = np.clip(result[:,:,0] * (1 + adjustment/2), 0, 255)  # Red
    return result.astype(np.uint8)

# Register these functions with the NL processor
nl_processor.register_function(
    "adjust_exposure",
    adjust_exposure,
    "Adjust the brightness/exposure of the image",
    {"amount": {"type": "number", "description": "Amount to adjust exposure (-1.0 to 1.0)"}}
)

nl_processor.register_function(
    "adjust_contrast",
    adjust_contrast,
    "Adjust the contrast of the image",
    {"multiplier": {"type": "number", "description": "Contrast multiplier (0.5 to 2.0)"}}
)

nl_processor.register_function(
    "adjust_saturation",
    adjust_saturation,
    "Adjust the color saturation of the image",
    {"adjustment": {"type": "number", "description": "Saturation adjustment (-1.0 to 1.0)"}}
)

nl_processor.register_function(
    "adjust_temperature",
    adjust_temperature,
    "Adjust the color temperature (warmth/coolness) of the image",
    {"adjustment": {"type": "number", "description": "Temperature adjustment (-0.5 to 0.5)"}}
)


Now let's try editing our image using natural language instructions.


In [ ]:
# Process a natural language instruction
instruction = "Make the image warmer and increase the contrast slightly"
print(f"Instruction: '{instruction}'")

# Process the instruction
processed_image, metadata = nl_processor.process(image, instruction)

# Display the functions that were called
print("\nFunctions called:")
for func_call in metadata['functions_called']:
    print(f"- {func_call['name']}({', '.join([f'{k}={v}' for k, v in func_call['args'].items()])})")

# Display before and after
display_before_after(image, processed_image, ["Original Image", f"After: '{instruction}'"])


Let's try a few more natural language instructions to see how the system responds to different requests.


In [ ]:
# Try different instructions
instructions = [
    "Make the colors more vibrant",
    "Make the image cooler and increase contrast",
    "Brighten the image and add warmth"
]

results = []
titles = ["Original"]

# Process each instruction
for instruction in instructions:
    processed, metadata = nl_processor.process(image, instruction)
    results.append(processed)
    titles.append(f"'{instruction}'")
    
    print(f"\nInstruction: '{instruction}'")
    print("Functions called:")
    for func_call in metadata['functions_called']:
        print(f"- {func_call['name']}({', '.join([f'{k}={v}' for k, v in func_call['args'].items()])})")

# Display all results
display_multiple([image] + results, titles)


## 3. RAG-Based Style Recommendations

Our photo editor also uses Retrieval Augmented Generation (RAG) to recommend cinematic styles based on image content. This combines a knowledge base of cinematography techniques with AI image analysis to suggest styles that match the content of the photo.

Let's see how this works.


In [ ]:
# Initialize the RAG style engine
rag_engine = RAGStyleEngine()

# Get style recommendations based on image content
recommendations = rag_engine.recommend_style(image)

# Display the recommendations
print("Style Recommendations Based on Image Content:")
for i, rec in enumerate(recommendations):
    print(f"\n{i+1}. {rec['style']} (Score: {rec['score']})")
    print(f"   Description: {rec['description']}")
    print(f"   Reasoning:")
    for reason in rec['reasoning']:
        print(f"   - {reason}")


Now, let's apply these recommended styles to our image and see the results.


In [ ]:
# Apply the recommended styles
styled_images = []
style_names = []

for rec in recommendations:
    style_name = rec['style']
    styled = rag_engine.apply_style(image, style_name)
    styled_images.append(styled)
    style_names.append(style_name)

# Display the original and styled images
display_multiple([image] + styled_images, ["Original"] + style_names)


### Style Recommendations Based on Description

We can also recommend styles based on a description provided by the user. This allows users to describe the look they want in natural language, and the system will find matching styles.


In [ ]:
# Get style recommendations based on a description
description = "I want a dramatic movie look with high contrast"
desc_recommendations = rag_engine.recommend_style(image, description)

# Display the recommendations
print(f"Style Recommendations Based on Description: '{description}'")
for i, rec in enumerate(desc_recommendations):
    print(f"\n{i+1}. {rec['style']} (Score: {rec['score']})")
    print(f"   Description: {rec['description']}")
    print(f"   Reasoning:")
    for reason in rec['reasoning']:
        print(f"   - {reason}")


Let's apply these description-based style recommendations.


In [ ]:
# Apply the description-based recommended styles
desc_styled_images = []
desc_style_names = []

for rec in desc_recommendations:
    style_name = rec['style']
    styled = rag_engine.apply_style(image, style_name)
    desc_styled_images.append(styled)
    desc_style_names.append(style_name)

# Display the original and styled images
display_multiple([image] + desc_styled_images, ["Original"] + desc_style_names)


## 4. Innovative Use Case: AI-Guided Creative Photography

Now let's explore an innovative use case that combines all of these AI capabilities: AI-guided creative photography. In this scenario, the AI analyzes an image, suggests creative directions, and helps the user achieve a specific artistic vision.

This approach is particularly valuable for:
- Amateur photographers looking to achieve professional results
- Creative professionals seeking inspiration
- Educators teaching photography and editing techniques

Let's see how this works in practice.


In [ ]:
def ai_guided_creative_workflow(image, creative_direction):
    """Demonstrate an AI-guided creative workflow.
    
    Args:
        image: Input image
        creative_direction: Description of the desired creative direction
        
    Returns:
        Tuple of (final image, workflow steps)
    """
    workflow_steps = []
    results = [image.copy()]
    
    # Step 1: Analyze the image
    ai_analyzer = AIImageAnalyzer()
    adjustments, analysis = ai_analyzer.analyze(image)
    
    workflow_steps.append({
        "step": "Image Analysis",
        "description": f"AI analyzed the image and identified it as a {analysis['scene_type']} scene with {analysis['lighting_condition']} lighting."
    })
    
    # Step 2: Apply basic adjustments
    basic_adjusted = apply_adjustments(image, adjustments)
    results.append(basic_adjusted)
    
    workflow_steps.append({
        "step": "Basic Adjustments",
        "description": f"Applied {len(adjustments)} AI-recommended adjustments to optimize the image."
    })
    
    # Step 3: Find styles matching the creative direction
    rag_engine = RAGStyleEngine()
    style_recs = rag_engine.recommend_style(image, creative_direction)
    
    if style_recs:
        # Apply the top recommended style
        top_style = style_recs[0]['style']
        styled_image = rag_engine.apply_style(basic_adjusted, top_style)
        results.append(styled_image)
        
        workflow_steps.append({
            "step": "Style Application",
            "description": f"Applied '{top_style}' style based on the creative direction: '{creative_direction}'"
        })
    
    # Step 4: Fine-tune with natural language processing
    nl_processor = NLProcessor()
    
    # Register the same functions as before
    nl_processor.register_function("adjust_exposure", adjust_exposure, 
                                 "Adjust the brightness/exposure of the image",
                                 {"amount": {"type": "number", "description": "Amount to adjust exposure (-1.0 to 1.0)"}})
    
    nl_processor.register_function("adjust_contrast", adjust_contrast,
                                 "Adjust the contrast of the image",
                                 {"multiplier": {"type": "number", "description": "Contrast multiplier (0.5 to 2.0)"}})
    
    nl_processor.register_function("adjust_saturation", adjust_saturation,
                                 "Adjust the color saturation of the image",
                                 {"adjustment": {"type": "number", "description": "Saturation adjustment (-1.0 to 1.0)"}})
    
    nl_processor.register_function("adjust_temperature", adjust_temperature,
                                 "Adjust the color temperature (warmth/coolness) of the image",
                                 {"adjustment": {"type": "number", "description": "Temperature adjustment (-0.5 to 0.5)"}})
    
    # Generate a fine-tuning instruction based on the creative direction
    fine_tuning_instruction = f"Fine-tune for {creative_direction}"
    final_image, metadata = nl_processor.process(results[-1], fine_tuning_instruction)
    results.append(final_image)
    
    workflow_steps.append({
        "step": "Fine-tuning",
        "description": f"Applied natural language fine-tuning: '{fine_tuning_instruction}'",
        "functions": [f"{func['name']}({', '.join([f'{k}={v}' for k, v in func['args'].items()])})" 
                     for func in metadata['functions_called']]
    })
    
    return final_image, results, workflow_steps


In [ ]:
# Try the AI-guided creative workflow with different creative directions
creative_directions = [
    "dramatic cinematic look",
    "warm vintage feel",
    "professional portrait style"
]

for direction in creative_directions:
    print(f"\n\n=== AI-Guided Creative Workflow: '{direction}' ===\n")
    
    final_image, workflow_images, steps = ai_guided_creative_workflow(image, direction)
    
    # Display the workflow steps
    for i, step in enumerate(steps):
        print(f"\nStep {i+1}: {step['step']}")
        print(f"  {step['description']}")
        if 'functions' in step:
            print("  Functions applied:")
            for func in step['functions']:
                print(f"  - {func}")
    
    # Display the workflow images
    step_titles = ["Original"] + [step["step"] for step in steps]
    display_multiple(workflow_images, step_titles)


## Conclusion

In this notebook, we've demonstrated how generative AI transforms the photo editing experience by making it more accessible, intelligent, and efficient. The key innovations include:

1. **AI-powered image analysis** that understands content and recommends appropriate adjustments
2. **Natural language photo editing** that allows users to describe what they want in plain English
3. **RAG-based style recommendations** that suggest cinematic styles based on image content and user descriptions
4. **AI-guided creative workflows** that combine these capabilities to help users achieve specific artistic visions

These capabilities democratize photo editing by removing the technical barriers that traditionally made it difficult for casual photographers to achieve professional-looking results. By understanding what's in the photo and what the user wants to achieve, the AI can guide them through the editing process and help them create images that match their creative vision.

The future of photo editing is not about replacing human creativity, but about augmenting it with AI that understands both the technical aspects of photography and the artistic intentions of the user.